# Get metadata from OMERO via JSON API
- Documentation https://docs.openmicroscopy.org/omero/5.6.0/developers/json-api.html 

In [ ]:
import os
import json
import requests
from getpass import getpass

In [ ]:
thisdir = os.path.abspath('')
interim_data = os.path.join(thisdir, '..\\data\\interim')

#### Set the server and the credentials

In [ ]:
server = ""
username = ""
password = ""

#### Create a session

In [ ]:
session = requests.Session()

#### Get the supported APIs as well as the connected base URLs

In [ ]:
r = session.get("http://" + server + "/api/").json()
r

#### We want the latest API's base url for working with the images

NOTE: '-1' is the latest api tuple in the returned json output. Read out the base url for further processing

In [ ]:
base_url = r["data"][-1]["url:base"]
base_url

#### Now get all URLs the server supports. We will pick the ones we need from the output.

In [ ]:
all_urls = session.get(base_url).json()
all_urls

#### Get the token for login

In [ ]:
token_url = all_urls["url:token"]
token = session.get(token_url).json()["data"]
token

#### Get the login URL

In [ ]:
login_url = all_urls["url:login"]
login_url

#### List of servers

In [ ]:
servers_url = all_urls["url:servers"]
servers = session.get(servers_url).json()["data"]
servers

#### Assume that there is only one server for now.

In [ ]:
server = servers[0]

#### Credentials for the login

In [ ]:
credentials = {"username": username,
               "password": password,
               "csrfmiddlewaretoken": token,
               "server": server["id"]}
credentials

#### Log in to the server and check if we succeeded

In [ ]:
return_value = session.post(login_url, data=credentials)
response = return_value.json()

assert return_value.status_code == 200
assert response["success"]

#### Get the list of projects we have access to

In [ ]:
projects_url = all_urls["url:projects"]
projects = session.get(projects_url).json()
projects

### Now get all available images 

In [ ]:
# get image URL
images_url = all_urls["url:images"]
images_url

In [ ]:
# get image data
images = session.get(images_url).json()

In [ ]:
# get json data
for i in images["data"]:
            image_url = i["url:image"]
            print(image_url)
            json_data = session.get(image_url).json()
            json_data.update({"ImageUrl": image_url})
            #f = open(json_data["data"]["Name"]+".json", "w")
            f = open(os.path.join(interim_data, json_data["data"]["Name"]+".json"), "w")
            f.write(json.dumps(json_data, indent=4, sort_keys=True))
            f.close()

#### Same as above but with loop over projects

In [ ]:
for p in projects["data"]:
    # Get the URL for the datasets of the project
    project_url = projects_url + str(p["@id"]) + "/"
    print(project_url)
    project = session.get(project_url).json()
    project_datasets_url = project["data"]["url:datasets"]
    print(project_datasets_url)
    # Get the datasets from the respective project
    project_datasets = session.get(project_datasets_url).json()
    # Get the images from the datasets
    for d in project_datasets["data"]:
        project_images_url = d["url:images"]
        print(project_images_url)
        project_images = session.get(project_images_url).json()
        # Write the metadata of each image into a json file
        for i in project_images["data"]:
            image_url = i["url:image"]
            print(image_url)
            json_data = session.get(image_url).json()
            #json_data.update({"ImageUrl": image_url})
            f = open(json_data["data"]["Name"]+".json", "w")
            f.write(json.dumps(json_data, indent=4, sort_keys=True))
            f.close()